In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arguments import Arguments
from cnn import CNN
from collections import defaultdict
from distributor import get_cluster_sizes, get_distributed_data, \
    get_distributed_data_using_loader, get_fog_graph
from train import fog_train as train
from numpy import array
from numpy.random import permutation, randint
import os
from sklearn.model_selection import train_test_split
import syft as sy
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from train import fog_train as train, test


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [3]:
# Setups
args = Arguments()
hook = sy.TorchHook(torch)
USE_CUDA = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if USE_CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
kwargs = {}

In [4]:
ckpt_path = '../ckpts'
dataset = 'mnist'
clf_type = 'cnn'
paradigm = 'fog'
model_name = '{}_{}_{}'.format(dataset, clf_type, paradigm)
init_path = os.path.join(ckpt_path, 'mnist_cnn_fl.init')
best_path = os.path.join(ckpt_path, model_name + '.best')
stop_path = os.path.join(ckpt_path, model_name + '.stop')

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.num_train, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

for data, target in train_loader:
    X_train = data
    y_train = target

for data, target in test_loader:
    X_test = data
    y_test = target

print('X_train: {}'.format(X_train.shape))
print('y_train: {}'.format(y_train.shape))

print('X_test: {}'.format(X_test.shape))
print('y_test: {}'.format(y_test.shape))

X_train: torch.Size([60000, 1, 28, 28])
y_train: torch.Size([60000])
X_test: torch.Size([10000, 1, 28, 28])
y_test: torch.Size([10000])


In [6]:
# prepare graph and data
fog_graph, workers = get_fog_graph(hook, args.num_workers, args.num_clusters,
                                 args.shuffle_workers, args.uniform_clusters)
X_trains, y_trains = get_distributed_data(X_train, y_train, args.num_workers)
# X_trains, y_trains = get_distributed_data_using_loader(train_loader)
fog_graph

{'L1_W0': ['L0_W25', 'L0_W20', 'L0_W42', 'L0_W27', 'L0_W17'],
 'L1_W1': ['L0_W48', 'L0_W34', 'L0_W6', 'L0_W45', 'L0_W31'],
 'L1_W2': ['L0_W13', 'L0_W15', 'L0_W10', 'L0_W28', 'L0_W0'],
 'L1_W3': ['L0_W43', 'L0_W4', 'L0_W7', 'L0_W35', 'L0_W11'],
 'L1_W4': ['L0_W22', 'L0_W1', 'L0_W33', 'L0_W41', 'L0_W49'],
 'L1_W5': ['L0_W38', 'L0_W36', 'L0_W40', 'L0_W14', 'L0_W5'],
 'L1_W6': ['L0_W18', 'L0_W44', 'L0_W12', 'L0_W30', 'L0_W32'],
 'L1_W7': ['L0_W37', 'L0_W47', 'L0_W8', 'L0_W24', 'L0_W3'],
 'L1_W8': ['L0_W26', 'L0_W29', 'L0_W2', 'L0_W23', 'L0_W46'],
 'L1_W9': ['L0_W19', 'L0_W16', 'L0_W39', 'L0_W21', 'L0_W9'],
 'L2_W0': ['L1_W8', 'L1_W5'],
 'L2_W1': ['L1_W3', 'L1_W0'],
 'L2_W2': ['L1_W1', 'L1_W6'],
 'L2_W3': ['L1_W7', 'L1_W4'],
 'L2_W4': ['L1_W2', 'L1_W9'],
 'L3_W0': ['L2_W0', 'L2_W3', 'L2_W1'],
 'L3_W1': ['L2_W4', 'L2_W2'],
 'L4_W0': ['L3_W1', 'L3_W0']}

In [9]:
best = 0
# Fire the engines
model = CNN().to(device)
if args.load_init:
    model.load_state_dict(torch.load(init_path))
    print('Load init: {}'.format(init_path))
elif args.save_init:
    torch.save(model.state_dict(), init_path)
    print('Save init: {}'.format(init_path))


Load init: ../ckpts/mnist_cnn_fl.init


In [10]:
best = 0

for epoch in range(1, args.epochs + 1):
    train(args, model, fog_graph, workers, X_trains, y_trains,
          device, epoch)
    acc = test(args, model, device, test_loader, best, epoch)

    if args.save_model and acc > best:
        best = acc
        torch.save(model.state_dict(), best_path)
        print('Model best  @ {}, acc {}: {}\n'.format(epoch, acc, best_path))
    
if (args.save_model):
    torch.save(model.state_dict(), stop_path)
    print('Model stop: {}'.format(stop_path))


Model best  @ 1, acc 0.1793: ../ckpts/mnist_cnn_fog.best



Model best  @ 2, acc 0.3031: ../ckpts/mnist_cnn_fog.best



Model best  @ 3, acc 0.3945: ../ckpts/mnist_cnn_fog.best



Model best  @ 4, acc 0.4855: ../ckpts/mnist_cnn_fog.best



Model best  @ 5, acc 0.5699: ../ckpts/mnist_cnn_fog.best



...

Model best  @ 189, acc 0.9631: ../ckpts/mnist_cnn_fog.best



Train Epoch: 190 	Loss: 0.142549 +- 0.017170



Test set: Average loss: 0.1321, Accuracy: 9591/10000 (95.91%) ==> 96.31%


Train Epoch: 200 	Loss: 1.102600 +- 0.025999



Test set: Average loss: 0.6552, Accuracy: 7808/10000 (78.08%) ==> 96.31%
Model stop: ../ckpts/mnist_cnn_fog.stop
